<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/q%26a/an%C3%A1lisis_de_archivos_pdf_con_chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.9/666.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 559.8 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [21]:
import os
import sys

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [27]:
path = '/content/gdrive/MyDrive/Colab Notebooks/Inteligencia Artificial/Q&A/'

In [28]:
sys.path.append(path)

In [29]:
from config import api_key_openai

In [30]:
os.environ["OPENAI_API_KEY"] = api_key_openai

In [34]:
reader = PdfReader('/content/gdrive/My Drive/Colab Notebooks/Inteligencia Artificial/Q&A/data/Barber_CardioSystem.pdf')

In [35]:
reader

In [36]:
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [55]:
print(raw_text[750:1505])

 
Barber Cardiosystems  
A mediados de enero de 2019, Ellie Barber -Collins, presidenta de Barber Cardiosystems (Barber), 
entró en la sala de conferencias adyacente a su oficina para reunirse con el equipo directivo superior 
de la empresa. En el primer lugar de la agenda del grupo figuraba qué había que hacer con dos de los 
componentes del sistema de remuneración de la empresa. Uno de los principales competidores de Barber había anunciado una reducción de precios del 10  % durante el verano, y los productos 
fabricados por nuevos comp etidores menos conocidos estaban entrando constantemente en el mercado 
y ejerciendo presión sobre los precios. Estas amenazas duales a las ventas de Barber amenazaban los márgenes y la participación de mercado.


In [56]:
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [57]:
len(texts)

38

In [62]:
texts[1]

'entró en la sala de conferencias adyacente a su oficina para reunirse con el equipo directivo superior \nde la empresa. En el primer lugar de la agenda del grupo figuraba qué había que hacer con dos de los \ncomponentes del sistema de remuneración de la empresa. Uno de los principales competidores de Barber había anunciado una reducción de precios del 10  % durante el verano, y los productos \nfabricados por nuevos comp etidores menos conocidos estaban entrando constantemente en el mercado \ny ejerciendo presión sobre los precios. Estas amenazas duales a las ventas de Barber amenazaban los márgenes y la participación de mercado. "Sé lo mucho que hemos trabajado para mantener los costos \nbajos, pero puede que hayamos llegado al límite de cuánto más podemos extraer de nuestras \noperaciones", dijo al grupo , mientras todos asentían con la cabeza. Mirando alrededor de la mesa,'

In [63]:
embeddings = OpenAIEmbeddings()

In [64]:
docsearch = FAISS.from_texts(texts, embeddings)

In [65]:
docsearch

In [66]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [67]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [68]:
query = "¿Quiénes son los autores del artículo?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Ranjay Gulati y Paul S. Myers'

In [69]:
query = "Analice el entorno en el que compite Barber ¿Cuáles son algunos de los desafíos en el mercado que enfrenta Barber?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Barber enfrenta desafíos en el mercado como amenazas duales a las ventas, como la reducción de precios del 10% anunciada por uno de sus principales competidores durante el verano, y los productos fabricados por nuevos competidores menos conocidos que están entrando constantemente en el mercado y ejerciendo presión sobre los precios. Esto amenaza los márgenes y la participación de mercado de Barber.'

In [70]:
query = "Analice las operaciones de Barber entre otras ventas, diseño/ desarrollo de producto, calidad, logista de entrega, producción/ productividad, inventario, control financiero. ¿las operaciones están alienadas con la estrategia y apoyan su cumplimiento?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Sí, las operaciones de Barber están alineadas con la estrategia y apoyan su cumplimiento. Por ejemplo, Barber gastaba alrededor de un 40% menos en investigación y desarrollo en 2018 que el promedio de la industria, lo que le permitió reducir sus gastos en I+D mientras se mantenía a la altura de los líderes del mercado. Además, la empresa estaba trabajando en el desarrollo de nuevos productos para ampliar su gama de productos y el enfoque de diseño integrado aceleraba el tiempo de desarrollo y reducía los costos al minimizar las fallas de los productos o del diseño y al optimizar los recursos. La empresa también ofrecía una bonificación mensual de hasta el 8% a los equipos de producción por superar los objetivos establecidos en términos de calidad y mej'

In [71]:
query = "¿Cuáles son los elementos de Responsabilidad Social Competitiva (RSC) de la empresa que detecta en el caso?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Barber Cardiosystems tiene un objetivo de dividendo anual del 30% de los ingresos netos, el 60% de los trabajadores participa en el plan voluntario de compra de acciones para empleados, trabajan para desarrollar nuevos productos para tratar afecciones cardíacas y exigen que se verifiquen todos los costos antes de aprobar su producción. Estos son algunos de los elementos de responsabilidad social competitiva de la empresa.'

In [72]:
query = "La estrategia de Barber Cardiosystems se centra en competir por costes. ¿Puede Barber continuar con éxito competir en precio, o necesita una estrategia basada más en la innovación y diferenciación?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nParece que Barber necesita una estrategia basada en la innovación y la diferenciación. Han estado trabajando para reducir costos y mejorar la eficiencia durante años, pero ahora hay una mayor presión de competidores con precios más bajos. Además, han reducido sus gastos en investigación y desarrollo, lo que indica que necesitan invertir más para poder seguir siendo competitivos.'

In [74]:
query = "¿Cuáles serían aglunas alternativas estratégicas cara al futuro de Barber distintas de la competencia en costes y explica detalladamente cómo?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Una estrategia alternativa para Barber podría ser aumentar la innovación para desarrollar productos mejores y más avanzados que los ofrecidos por sus competidores. Esto permitiría a Barber incrementar los precios y tratar de recuperar los márgenes de ganancia. Esto implicaría una mayor inversión en I+D para mejorar los productos existentes y desarrollar nuevos productos, así como una mayor inversión en la publicidad para promocionar los nuevos productos.'

**Por hacer**

* Añadir explicación general (objetivo)
* Incluir referencias